In [1]:
!pip install plotly openpyxl xlsxwriter pandasql --quiet

import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import os, math

csv_path = '/content/Sample - Superstore.csv'
assert os.path.exists(csv_path), f"CSV file not found at {csv_path}"

# 1) Load dataset
df = pd.read_csv(csv_path, encoding='latin1')
print("Loaded dataset with shape:", df.shape)
print("Columns:", df.columns.tolist())

# Quick preview
display(df.head())

# 2) Clean & preprocess
df_clean = df.copy()
df_clean.columns = [c.strip() for c in df_clean.columns]  # tidy headers
df_clean.dropna(how='all', inplace=True)
df_clean = df_clean.drop_duplicates()

# Identify standard columns (works for common 'Sample - Superstore' layout)
date_col = next((c for c in df_clean.columns if 'order' in c.lower() and 'date' in c.lower()), None)
sales_col = next((c for c in df_clean.columns if 'sales' == c.lower() or 'sales' in c.lower()), None)
qty_col = next((c for c in df_clean.columns if 'quantity' in c.lower() or 'qty' in c.lower()), None)
profit_col = next((c for c in df_clean.columns if 'profit' in c.lower()), None)
category_col = next((c for c in df_clean.columns if 'category' in c.lower()), None) \
               or next((c for c in df_clean.columns if 'sub-category' in c.lower()), None) \
               or next((c for c in df_clean.columns if 'product' in c.lower()), None)
region_col = next((c for c in df_clean.columns if 'region' in c.lower()), None)
customer_col = next((c for c in df_clean.columns if 'customer id' in c.lower() or 'customer' == c.lower() or 'customer name' in c.lower()), None)
product_name_col = next((c for c in df_clean.columns if 'product name' in c.lower() or 'product'==c.lower()), None)

print("Detected:", {
    'date_col': date_col,
    'sales_col': sales_col,
    'qty_col': qty_col,
    'profit_col': profit_col,
    'category_col': category_col,
    'region_col': region_col,
    'customer_col': customer_col,
    'product_name_col': product_name_col
})

# Parse dates
if date_col:
    df_clean[date_col] = pd.to_datetime(df_clean[date_col], errors='coerce')

# Ensure numeric columns
if sales_col:
    df_clean[sales_col] = pd.to_numeric(df_clean[sales_col], errors='coerce')
if qty_col:
    df_clean[qty_col] = pd.to_numeric(df_clean[qty_col], errors='coerce')
if profit_col:
    df_clean[profit_col] = pd.to_numeric(df_clean[profit_col], errors='coerce')

# Drop rows that cannot be used for revenue/time analysis (missing sales or dates)
critical_cols = [c for c in [sales_col, date_col] if c]
if critical_cols:
    before = df_clean.shape[0]
    df_clean = df_clean.dropna(subset=critical_cols)
    after = df_clean.shape[0]
    print(f"Dropped {before-after} rows missing critical columns {critical_cols}")

# Fill numeric NA with 0 for non-critical numeric cols
numeric_cols = df_clean.select_dtypes(include=['number']).columns.tolist()
df_clean[numeric_cols] = df_clean[numeric_cols].fillna(0)

# Trim string columns
str_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
df_clean[str_cols] = df_clean[str_cols].apply(lambda s: s.str.strip())

# Derived time features
if date_col:
    df_clean['order_month'] = df_clean[date_col].dt.to_period('M').dt.to_timestamp()
    df_clean['order_year'] = df_clean[date_col].dt.year
else:
    df_clean['order_month'] = pd.NaT
    df_clean['order_year'] = np.nan

# Ensure qty and profit exist
if qty_col is None:
    df_clean['Quantity'] = df_clean.get('Quantity', 1).fillna(1)
    qty_col = 'Quantity'
if profit_col is None:
    df_clean['Profit'] = df_clean.get('Profit', 0).fillna(0)
    profit_col = 'Profit'

# 3) KPIs and summarizations
total_revenue = df_clean[sales_col].sum()
total_orders = df_clean.shape[0]
average_order_value = total_revenue / total_orders if total_orders else 0
total_profit = df_clean[profit_col].sum()

kpis = {
    'Total Revenue': total_revenue,
    'Total Orders': total_orders,
    'Average Order Value': average_order_value,
    'Total Profit': total_profit
}
print("\nKey KPIs")
for k,v in kpis.items():
    print(f" - {k}: {v:,.2f}")

# Revenue by category
if category_col and category_col in df_clean.columns:
    revenue_by_category = df_clean.groupby(category_col)[sales_col].sum().reset_index().sort_values(sales_col, ascending=False)
else:
    revenue_by_category = pd.DataFrame(columns=[category_col, sales_col])

# Sales by region
if region_col and region_col in df_clean.columns:
    sales_by_region = df_clean.groupby(region_col)[sales_col].sum().reset_index().sort_values(sales_col, ascending=False)
else:
    sales_by_region = pd.DataFrame(columns=[region_col, sales_col])

# Monthly sales trend
if 'order_month' in df_clean.columns:
    monthly_sales = df_clean.groupby('order_month')[sales_col].sum().reset_index().sort_values('order_month')
else:
    monthly_sales = pd.DataFrame(columns=['order_month', sales_col])

# Top products
if product_name_col and product_name_col in df_clean.columns:
    top_products = df_clean.groupby(product_name_col)[sales_col].sum().reset_index().sort_values(sales_col, ascending=False)
else:
    top_products = pd.DataFrame(columns=[product_name_col, sales_col])

# 4) Customer segments (simple RFM-like), robust to duplicates/ties
customer_segments = pd.DataFrame()
if customer_col and date_col and customer_col in df_clean.columns:
    ref_date = df_clean[date_col].max()
    rfm = df_clean.groupby(customer_col).agg({
        date_col: lambda x: (ref_date - x.max()).days,
        sales_col: 'sum',
        qty_col: 'sum'
    }).rename(columns={date_col: 'recency', sales_col: 'monetary', qty_col: 'frequency'}).reset_index()

    # For scoring, handle constant columns or ties safely
    def safe_qcut(series, q=4, labels=None, ascending=True):
        try:
            if labels is None:
                labels = range(1, q+1)
            if not ascending:
                # reverse ranks when higher value = better (monetary, frequency)
                return pd.qcut(series.rank(method='first', ascending=False), q, labels=labels)
            else:
                return pd.qcut(series.rank(method='first', ascending=True), q, labels=labels)
        except Exception:
            # fallback: use rank percentile bins
            ranks = series.rank(method='first', pct=True)
            bins = pd.cut(ranks, bins=q, labels=labels, include_lowest=True)
            return bins

    rfm['r_score'] = safe_qcut(rfm['recency'], q=4, labels=[4,3,2,1], ascending=True)  # lower recency (recent) => higher score
    rfm['m_score'] = safe_qcut(rfm['monetary'], q=4, labels=[1,2,3,4], ascending=False) # higher monetary => higher score
    rfm['f_score'] = safe_qcut(rfm['frequency'], q=4, labels=[1,2,3,4], ascending=False)

    # Combine to segment code and categorize high/mid/low
    rfm['r_score'] = rfm['r_score'].astype(int)
    rfm['m_score'] = rfm['m_score'].astype(int)
    rfm['f_score'] = rfm['f_score'].astype(int)
    rfm['segment_score'] = rfm['r_score'].astype(str) + rfm['m_score'].astype(str) + rfm['f_score'].astype(str)

    median_m = rfm['monetary'].median() if not rfm['monetary'].empty else 0
    q3_m = rfm['monetary'].quantile(0.75) if not rfm['monetary'].empty else 0
    rfm['segment'] = pd.cut(rfm['monetary'], bins=[-1, median_m, q3_m, rfm['monetary'].max()+1], labels=['Low Value','Mid Value','High Value'])
    customer_segments = rfm.sort_values('monetary', ascending=False)
else:
    customer_segments = pd.DataFrame()

# 5) Example SQL queries (sqlite in-memory)
conn = sqlite3.connect(':memory:')
df_clean.to_sql('sales', conn, index=False, if_exists='replace')

sql_examples = {
    'total_revenue': f"SELECT SUM([{sales_col}]) as total_revenue FROM sales;",
    'revenue_by_category': f"SELECT [{category_col}] as category, SUM([{sales_col}]) as revenue FROM sales GROUP BY [{category_col}] ORDER BY revenue DESC LIMIT 20;" if category_col else None,
    'sales_by_region': f"SELECT [{region_col}] as region, SUM([{sales_col}]) as revenue FROM sales GROUP BY [{region_col}] ORDER BY revenue DESC;" if region_col else None,
    'monthly_sales': f"SELECT order_month, SUM([{sales_col}]) as revenue FROM sales GROUP BY order_month ORDER BY order_month;"
}

print("\nSample SQL queries (you can run them with conn.execute(...).fetchall())")
for k,q in sql_examples.items():
    if q:
        print(f"- {k}:\n  {q}")

# 6) Interactive Plotly charts (display inline in Colab)
print("\nRendering interactive charts...")

# KPI indicators
fig_kpis = go.Figure()
fig_kpis.add_trace(go.Indicator(mode="number", value=total_revenue, title={"text":"Total Revenue"}))
fig_kpis.add_trace(go.Indicator(mode="number", value=average_order_value, title={"text":"Average Order Value"}))
fig_kpis.add_trace(go.Indicator(mode="number", value=total_orders, title={"text":"Total Orders"}))
fig_kpis.update_layout(grid={'rows':1,'columns':3}, height=300, margin=dict(l=10,r=10,t=40,b=10))
fig_kpis.show()

# Monthly sales (if available)
if not monthly_sales.empty:
    fig_monthly = px.line(monthly_sales, x='order_month', y=sales_col, title='Monthly Sales Trend', markers=True)
    fig_monthly.update_xaxes(rangeslider_visible=True)
    fig_monthly.show()
else:
    print("No monthly sales data to plot (missing order dates).")

# Revenue by category bar
if not revenue_by_category.empty:
    fig_cat = px.bar(revenue_by_category.head(20), x=category_col, y=sales_col, title='Revenue by Category (Top 20)')
    fig_cat.update_layout(xaxis_tickangle=-45)
    fig_cat.show()

# Sales by region pie
if not sales_by_region.empty:
    fig_reg = px.pie(sales_by_region, names=region_col, values=sales_col, title='Sales by Region')
    fig_reg.show()

# Top products
if not top_products.empty:
    fig_top = px.bar(top_products.head(10), x=product_name_col, y=sales_col, title='Top Products by Revenue (Top 10)')
    fig_top.update_layout(xaxis_tickangle=-45)
    fig_top.show()

# 7) Export to Excel workbook with multiple sheets
excel_path = 'sales_dashboard.xlsx'
with pd.ExcelWriter(excel_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
    df_clean.to_excel(writer, sheet_name='cleaned_data', index=False)
    pd.DataFrame(list(kpis.items()), columns=['KPI','Value']).to_excel(writer, sheet_name='KPIs', index=False)
    revenue_by_category.to_excel(writer, sheet_name='Revenue_by_Category', index=False)
    sales_by_region.to_excel(writer, sheet_name='Sales_by_Region', index=False)
    monthly_sales.to_excel(writer, sheet_name='Monthly_Sales', index=False)
    top_products.to_excel(writer, sheet_name='Top_Products', index=False)
    customer_segments.to_excel(writer, sheet_name='Customer_Segments', index=False)
print(f"\nExcel workbook saved to: {excel_path}")

# 8) Automated insights & recommendations (simple heuristics)
print("\nAutomated Insights & Recommendations:")
insights = []
if not revenue_by_category.empty:
    top_cat = revenue_by_category.iloc[0][category_col]
    insights.append(f"Top-performing category: {top_cat} — consider promotions and additional SKUs in this category.")
if not sales_by_region.empty:
    top_reg = sales_by_region.iloc[0][region_col]
    insights.append(f"Top-selling region: {top_reg} — consider targeted marketing and inventory prioritization.")
if average_order_value > 0:
    insights.append(f"AOV is {average_order_value:,.2f}. Consider bundling or free-shipping thresholds to raise AOV.")
if total_profit and total_revenue and (total_profit / total_revenue) < 0.10:
    insights.append("Profit margin is relatively low (<10%) — review discounts, costs, and pricing.")
if not top_products.empty:
    top5 = top_products.head(5)[product_name_col].tolist()
    insights.append(f"Top 5 products: {', '.join(map(str, top5))} — prioritize cross-sell and replenishment for these SKUs.")

for s in insights:
    print(" -", s)

print("\nAll done. Files generated:")
print(" - Excel dashboard:", excel_path)


  Preparing metadata (setup.py) ... done
Loaded dataset with shape: (9994, 21)
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


Detected: {'date_col': 'Order Date', 'sales_col': 'Sales', 'qty_col': 'Quantity', 'profit_col': 'Profit', 'category_col': 'Category', 'region_col': 'Region', 'customer_col': 'Customer ID', 'product_name_col': 'Product Name'}
Dropped 0 rows missing critical columns ['Sales', 'Order Date']

Key KPIs
 - Total Revenue: 2,297,200.86
 - Total Orders: 9,994.00
 - Average Order Value: 229.86
 - Total Profit: 286,397.02

Sample SQL queries (you can run them with conn.execute(...).fetchall())
- total_revenue:
  SELECT SUM([Sales]) as total_revenue FROM sales;
- revenue_by_category:
  SELECT [Category] as category, SUM([Sales]) as revenue FROM sales GROUP BY [Category] ORDER BY revenue DESC LIMIT 20;
- sales_by_region:
  SELECT [Region] as region, SUM([Sales]) as revenue FROM sales GROUP BY [Region] ORDER BY revenue DESC;
- monthly_sales:
  SELECT order_month, SUM([Sales]) as revenue FROM sales GROUP BY order_month ORDER BY order_month;

Rendering interactive charts...



Excel workbook saved to: sales_dashboard.xlsx

Automated Insights & Recommendations:
 - Top-performing category: Technology — consider promotions and additional SKUs in this category.
 - Top-selling region: West — consider targeted marketing and inventory prioritization.
 - AOV is 229.86. Consider bundling or free-shipping thresholds to raise AOV.
 - Top 5 products: Canon imageCLASS 2200 Advanced Copier, Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind, Cisco TelePresence System EX90 Videoconferencing Unit, HON 5400 Series Task Chairs for Big and Tall, GBC DocuBind TL300 Electric Binding System — prioritize cross-sell and replenishment for these SKUs.

All done. Files generated:
 - Excel dashboard: sales_dashboard.xlsx
